## Parameter estimation: Normal

Parameter estimation is the strongest method of VaR estimation because it assumes that the loss distribution class is known. Parameters are estimated to fit data to this distribution, and statistical inference is then made.

In [1]:
import warnings
warnings.filterwarnings('ignore')

from pandas_datareader import DataReader
from datetime import datetime

stocklist = ['C', 'MS', 'GS', 'JPM']

start = datetime(2004,12,31)
end = datetime(2010,12,31)

portfolio = DataReader(stocklist, 'yahoo',start, end)['Close']
portfolio.rename(columns={'C':'Citibank', 'MS':'Morgan Stanley','GS':'Goldman Sachs','JPM':'J.P. Morgan'}, inplace=True)
print(portfolio.head())

weights = [0.25, 0.25, 0.25, 0.25]

Symbols       Citibank  Morgan Stanley  Goldman Sachs  J.P. Morgan
Date                                                              
2004-12-31  481.799988       55.520000     104.040001    39.009998
2005-01-03  482.700012       55.900002     104.949997    39.150002
2005-01-04  478.600006       55.299999     104.269997    38.410000
2005-01-05  484.600006       54.980000     103.800003    38.490002
2005-01-06  489.299988       56.279999     105.230003    38.709999


In [2]:
# Compute the portfolio's daily returns
portfolio_returns = portfolio.pct_change().dot(weights).dropna()

# Compute the portfolio's daily losses
portfolio_losses = -portfolio_returns.dropna()

In [3]:
# Import the Normal distribution and skewness test from scipy.stats
from scipy.stats import norm, anderson

# Fit portfolio losses to the Normal distribution
params = norm.fit(portfolio_losses)

# Compute the 95% VaR from the fitted distribution, using parameter estimates
VaR_95 = norm.ppf(0.95, *params)
print("VaR_95, Normal distribution: ", VaR_95)

# Test the data for Normality
print("Anderson-Darling test result: ", anderson(portfolio_losses))

VaR_95, Normal distribution:  0.053938106880521725
Anderson-Darling test result:  AndersonResult(statistic=86.57275579677253, critical_values=array([0.574, 0.654, 0.785, 0.916, 1.089]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]))


The Anderson-Darling test value of 86.57 exceeds the 99% critical value of 1.089 by a large margin, indicating that the Normal distribution may be a poor choice to represent portfolio losses.

## Parameter estimation: Skewed Normal

We'll parametrically estimate the 95% VaR of a loss distribution fit. This is a more general distribution than the Normal and allows losses to be non-symmetrically distributed. We might expect losses to be skewed during the crisis, when portfolio losses were more likely than gains.

In [4]:
# Import the skew-normal distribution and skewness test from scipy.stats
from scipy.stats import skewnorm, skewtest

# Fit the Student's t distribution to crisis losses
crisis_losses = portfolio_losses.loc['2008-01-01':'2009-12-31']

# Test the data for skewness
print("Skewtest result: ", skewtest(crisis_losses))

# Fit the portfolio loss data to the skew-normal distribution
params = skewnorm.fit(crisis_losses)

# Compute the 95% VaR from the fitted distribution, using parameter estimates
VaR_95 = skewnorm.ppf(0.95, *params)
print("VaR_95 from skew-normal: ", VaR_95)

Skewtest result:  SkewtestResult(statistic=-9.46377341649772, pvalue=2.9702650510105885e-21)
VaR_95 from skew-normal:  0.07953135436049741


The Anderson-Darling and skewtest results show the Normal distribution estimates cannot be relied upon. Skewness matters for loss distributions, and parameter estimation is one way to quantify this important feature of the financial crisis.

## Historical Simulation


Historical simulation of VaR assumes that the distribution of historical losses is the same as the distribution of future losses. We'll test if this is true for our investment bank portfolio by comparing the 95% VaR from 2005 - 2006 to the 95% VaR from 2007 - 2009.

In [5]:
import numpy as np

asset_returns_list = [portfolio['2005-01-01':'2006-12-31'].pct_change().dropna(), portfolio['2007-01-01':'2008-12-31'].pct_change().dropna()]

# Create portfolio returns for the two sub-periods using the list of asset returns
portfolio_returns_list = np.array([ x.dot(weights) for x in asset_returns_list])

# Derive portfolio losses from portfolio returns
losses = - portfolio_returns_list

# Find the historical simulated VaR estimates
VaR_95 = [np.quantile(x, 0.95) for x in losses]

# Display the VaR estimates
print("VaR_95, 2005-2006: ", VaR_95[0], '; VaR_95, 2007-2009: ', VaR_95[1])

VaR_95, 2005-2006:  0.01470023008198797 ; VaR_95, 2007-2009:  0.05492612537506693


The VaR estimates are very different for the two time periods. This indicates that over the entire 2005 - 2009 period the loss distribution was likely not stationary. Historical simulation, while very general, should be used with caution when the data is not from a stationary distribution.

## Monte Carlo Simulation

We can use Monte Carlo simulation of the 2005-2010 investment bank portfolio assets to find the 95% VaR.

In [6]:
from pypfopt.risk_models import CovarianceShrinkage

# Create the CovarianceShrinkage instance variable
cs = CovarianceShrinkage(portfolio)

# Compute the efficient covariance matrix of returns
e_cov = cs.ledoit_wolf()/252
e_cov = e_cov.to_numpy()

# Compute the mean and volatility
mu = -portfolio.pct_change().dropna().mean()
sigma = -portfolio.pct_change().dropna().std()

total_steps = 1440
N = 10000

In [8]:
# Initialize daily cumulative loss for the assets, across N runs
daily_loss = np.zeros((4,N))

# Create the Monte Carlo simulations for N runs
for n in range(N):
    a = norm.rvs(size = (4,total_steps))
    # Compute simulated path of length total_steps for correlated returns
    correlated_randomness = e_cov @ a
    # Adjust simulated path by total_steps and mean of portfolio losses
    steps = 1/total_steps
    minute_losses = mu * steps + correlated_randomness * np.sqrt(steps)
    daily_loss[:, n] = minute_losses.sum(axis=1)
    
# Generate the 95% VaR estimate
losses = weights @ daily_loss
print("Monte Carlo VaR_95 estimate: ", np.quantile(losses, 0.95))

ValueError: operands could not be broadcast together with shapes (4,) (4,1440) 

In [ ]:
a = norm.rvs(size = (4,total_steps))
for n in range(N):
    print(e_cov @ a)

[[-0.00184095 -0.00131664 -0.00345853 ... -0.00316408 -0.00197501
  -0.0011217 ]
 [-0.00271439 -0.00046892 -0.00422518 ... -0.00089798  0.00016285
  -0.00171992]
 [-0.0013186  -0.00013544 -0.0025709  ... -0.00182018 -0.00061226
  -0.00159547]
 [-0.0017863  -0.0002967  -0.00239283 ... -0.00229743 -0.0011753
  -0.00122335]]
[[-0.00184095 -0.00131664 -0.00345853 ... -0.00316408 -0.00197501
  -0.0011217 ]
 [-0.00271439 -0.00046892 -0.00422518 ... -0.00089798  0.00016285
  -0.00171992]
 [-0.0013186  -0.00013544 -0.0025709  ... -0.00182018 -0.00061226
  -0.00159547]
 [-0.0017863  -0.0002967  -0.00239283 ... -0.00229743 -0.0011753
  -0.00122335]]
[[-0.00184095 -0.00131664 -0.00345853 ... -0.00316408 -0.00197501
  -0.0011217 ]
 [-0.00271439 -0.00046892 -0.00422518 ... -0.00089798  0.00016285
  -0.00171992]
 [-0.0013186  -0.00013544 -0.0025709  ... -0.00182018 -0.00061226
  -0.00159547]
 [-0.0017863  -0.0002967  -0.00239283 ... -0.00229743 -0.0011753
  -0.00122335]]
[[-0.00184095 -0.00131664 -0